In [63]:
from bs4 import BeautifulSoup
import html.parser
import json
import csv
import time
import re
import requests

In [64]:
h = html.parser

In [5]:
urls = []
with open('urls_90.csv', 'r' ) as csv_file:
    csv_reader = csv.reader(csv_file)
    for line in csv_reader:
        url = str(line[0]).strip()
        urls.append(url)
print(len(urls))

6119


In [3]:
print(urls[152])

https://www.e-jumbo.gr/axesouar-i-y-kiniton/axesouar-gaming/gaming-headset-me-led-i-jmb_308564/


In [79]:
# for i in range(0,len(urls)):
url = "https://www.e-jumbo.gr/vrefika-eidi/vrefanaptyxi/peripoiisi-stithous-miteras/esoroucha-egkymosynis/soutien-egkymosynis-nude-lefki-dantela_1445696/"

headers = {
    'authority': 'www.e-jumbo.gr',
    'sec-ch-ua': '" Not;A Brand";v="99", "Google Chrome";v="91", "Chromium";v="91"',
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'x-requested-with': 'XMLHttpRequest',
    'sec-ch-ua-mobile': '?0',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.101 Safari/537.36',
    'content-type': 'application/json; charset=UTF-8',
    'origin': 'https://www.e-jumbo.gr',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'cors',
    'sec-fetch-dest': 'empty',
    'referer': url,
    'accept-language': 'en-US,en;q=0.9,hi;q=0.8',
}

params = (
    ('lang', 'el'),
)


r_1 = requests.get(url, headers=headers)
soup = BeautifulSoup(r_1.text, 'html.parser')

try:
    name = str(soup.select_one('h1[itemprop="name"]').text).strip()
except:
    pass

try:
    price = str(soup.select_one('[class="current-price"]').text).strip()
except:
    price = ""

try:    
    sku  = str(soup.select_one('[class="code"]').text).replace("Κωδικός Jumbo:","").strip()
except:
    sku  = ""

try:
    category = ""
    cats = soup.select('[class="breadcrumb clearfix"] [class="title"]')
    for cat in cats[:4]:
        category += str(cat.text).strip() + " > "
    category = category.strip(" > ")
except:
    category = ""

try:
    images = ""
    images_raw=soup.select('a[class="cloud-zoom"]')
    for img in images_raw:
        images += "https:" + str(img["href"]).strip().split(".jpg?")[0] + ".jpg" + "\n"
    images = images.strip("\n") 
    #print(images)
except:
    images = ""

try:
    description = ""
    sections = soup.select('[class="tabs-content"] [class*="tab-box"]')
    for sec in sections:
        label = str(sec.select_one('[class="title"]').text).strip()
        if label =="Περιγραφή - Χαρακτηριστικά":
            description = str(sec.select_one('[class="dynamic-content"]').text).strip()
except:
    description = ""

f_data = [url,name,description,sku,"","",images,price,category]

with open('output_90.csv', 'a', newline='', encoding="utf-8", errors="ignore") as new_file:
    csv_writer = csv.writer(new_file)
    csv_writer.writerow(f_data)    

# print(i)

versions = soup.select('[class="versions-container"] [class*="x-product-details-version-selector"] li')
if len(versions)==0:

    skuId = str(soup.select_one('[id="search-warehouse-wrapper"]')["data-skuid"]).strip()

    data = {"skuId":int(skuId),"skuCode":int(sku),"countryCode":"CY","zipCode":"","maxDistance":1000000,"ignoreAvailability":'true',"geoLocationWithZipCode":'true',"topAvailableWarehouse":100000,"hasGeolocation":'false',"latitude":0,"longitude":0}
    data = json.dumps(data)
    #print(data)

    r_2 = requests.post('https://www.e-jumbo.gr/services/EcomService.svc/GetProductAvailableWarehouses', headers=headers, params=params, data=data)
    w_data = json.loads(r_2.json()["d"]["data"])

    availability = "Μη διαθέσιμο" 

    try:
        warehouses   = w_data["WarehousesGrouped"][0]["Items"]
    except:
        warehouses   =  []
        availability = "διαθέσιμη στο διαδίκτυο"

    for wh in warehouses:
        area = str(wh["Area"]).strip()
        if area == "Λάρνακα":
            availability = str(wh["AvailabilityName"]).strip()

    f_data = ["","","","","None",availability,"","",""]

    with open('output_90.csv', 'a', newline='', encoding="utf-8", errors="ignore") as new_file:
        csv_writer = csv.writer(new_file)
        csv_writer.writerow(f_data)

#     print(i,r_1.status_code,r_2.status_code)

for version in versions:
    size  = str(version.select_one('[class="size"]').text).strip()
    skuId = str(version["data-value"]).strip()

    data = {"skuId":int(skuId),"skuCode":2538,"countryCode":"CY","zipCode":"","maxDistance":1000000,"ignoreAvailability":'true',"geoLocationWithZipCode":'true',"topAvailableWarehouse":100000,"hasGeolocation":'false',"latitude":0,"longitude":0}
    data = json.dumps(data)
    #print(data)

    r_2 = requests.post('https://www.e-jumbo.gr/services/EcomService.svc/GetProductAvailableWarehouses', headers=headers, params=params, data=data)
    w_data = json.loads(r_2.json()["d"]["data"])

    availability = "Μη διαθέσιμο" 

    try:
        warehouses   = w_data["WarehousesGrouped"][0]["Items"]
    except:
        warehouses   =  []
        availability = "διαθέσιμη στο διαδίκτυο"

    for wh in warehouses:
        area = str(wh["Area"]).strip()
        if area == "Λάρνακα":
            availability = str(wh["AvailabilityName"]).strip()

    f_data = ["","","","",size,availability,"","",""]

    with open('output_90.csv', 'a', newline='', encoding="utf-8", errors="ignore") as new_file:
        csv_writer = csv.writer(new_file)
        csv_writer.writerow(f_data)

    print(size,availability)
    break

85C διαθέσιμη στο διαδίκτυο


In [86]:
url = "https://www.e-jumbo.gr/vrefika-eidi/vrefanaptyxi/peripoiisi-stithous-miteras/esoroucha-egkymosynis/soutien-egkymosynis-nude-lefki-dantela_1445696/"

headers = {
    'authority': 'www.e-jumbo.gr',
    'sec-ch-ua': '" Not;A Brand";v="99", "Google Chrome";v="91", "Chromium";v="91"',
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'x-requested-with': 'XMLHttpRequest',
    'sec-ch-ua-mobile': '?0',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.101 Safari/537.36',
    'content-type': 'application/json; charset=UTF-8',
    'origin': 'https://www.e-jumbo.gr',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'cors',
    'sec-fetch-dest': 'empty',
    'referer': url,
    'accept-language': 'en-US,en;q=0.9,hi;q=0.8',
}

params = (
    ('lang', 'el'),
)


r_1 = requests.get(url, headers=headers)
soup = BeautifulSoup(r_1.text, 'html.parser')
p_details = soup.select_one('article[class*="product-details"]')
# print(p_details)
data_parent_id = str(p_details["data-parent-id"]).strip()
data_path      = str(p_details["data-path"]).strip()
print(data_path)

526038316


In [15]:
r_2.json()["d"]

{'__type': 'ServiceResponseOfCommonResponseTypeWH_P1lev_S:#Lighthouse.Lh3.FrontEnd.Services.Endpoints',
 'code': 'OperationSuccesful',
 'data': '{"StoresText":"ΚΑΤΑΣΤΗΜΑ","AvailabilityText":"ΔΙΑΘΕΣΙΜΟΤΗΤΑ","NoWarehouseFound":"Δεν βρέθηκαν καταστήματα","WarehousesGrouped":[]}',
 'email': None,
 'success': True,
 'time': 0}

In [179]:
import requests

headers = {
    'authority': 'www.e-jumbo.gr',
    'sec-ch-ua': '" Not;A Brand";v="99", "Google Chrome";v="91", "Chromium";v="91"',
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'x-requested-with': 'XMLHttpRequest',
    'sec-ch-ua-mobile': '?0',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.106 Safari/537.36',
    'content-type': 'application/json; charset=UTF-8',
    'origin': 'https://www.e-jumbo.gr',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'cors',
    'sec-fetch-dest': 'empty',
    'referer': 'https://www.e-jumbo.gr/vrefika-eidi/vrefanaptyxi/peripoiisi-stithous-miteras/esoroucha-egkymosynis/soutien-egkymosynis-nude-lefki-dantela_1445696/?country=GR',
    'accept-language': 'en-US,en;q=0.9,hi;q=0.8',
}

params = (
    ('lang', 'el'),
    ('country', 'cy'),
)


data = {}

d_value_data = "{\"__type\":\"Lighthouse.Lh3.FrontEnd.Root.Model.AdapterParameters, Vendd.Website.Backend, Version=2020.4.0.72, Culture=neutral, PublicKeyToken=null\",\"LoaderOptions\":{\"__type\":\"Lighthouse.Lh3.FrontEnd.Controls.Builtin.ProductDetailsLoaderOptions, Vendd.Website.Backend, Version=2020.4.0.72, Culture=neutral, PublicKeyToken=null\",\"ParentProductId\":1445696,\"PathChecksum\":526038316,\"LanguageId\":1,\"SkuId\":1388210,\"SplitDimensions\":null,\"BundleItems\":[],\"ItemAttributes\":[],\"LanguageCode\":\"el\",\"CurrencyCode\":\"EUR\",\"SiteCode\":\"www-e-jumbo-gr\",\"Cacher\":null},\"AdapterOptions\":{\"__type\":\"Lighthouse.Lh3.FrontEnd.Root.Model.ProductDetailsBaseAdapter+ProductDetailsAdapterParams, Vendd.Website.Backend, Version=2020.4.0.72, Culture=neutral, PublicKeyToken=null\",\"IsDynamic\":false,\"EnableNoImage\":true,\"DimensionImageTypes\":[],\"SelectDefaultSku\":false,\"Actions\":2147483647,\"LazyLoadPrices\":false,\"AttributesStartIndex\":null,\"Articles\":{},\"Literals\":{\"QuantityText\":\"ΠΟΣΟΤΗΤΑ:\",\"VatText\":\"Στην τιμή συμπεριλαμβάνεται ο Φ.Π.A.\",\"ShopOnlyProductBtnText\":\"ΕΠΙΚΟΙΝΩΝΗΣΤΕ ΜΕ ΚΑΤΑΣΤΗΜΑ\",\"ShopOnlyProductBtnText2\":\"βρες το πλησιέστερο\",\"StoresLocatorUrl\":\"http://corporate.e-jumbo.gr/katastimata-jumbo/\",\"ShopOnlyProductBtnTextUrl\":\"εδώ\",\"IsBatteryRequiredText\":\"ΛΕΙΤΟΥΡΓΕΙ ΜΕ ΜΠΑΤΑΡΙΕΣ\",\"BatteryBuyText\":\"ΔΕΙΤΕ ΕΔΩ\",\"IsBatteryRequiredUrl\":\"/paichnidia/bataries-alkalikes/\",\"SafeProductTitle\":\"ΚΑΤΑΛΛΗΛΟΤΗΤΑ ΠΡΟΪΟΝΤΟΣ\",\"PriceText\":\"ΤΙΜΗ JUMBO:\",\"AvailabilityText\":\"Διαθεσιμότητα:\",\"OrderedWithIconClass\":\"rocket\",\"GalleryText\":\"Τοποθετήστε τον κέρσορα στη φωτογραφία για μεγέθυνση\",\"AssortmentCodesText\":\"Το προϊόν είναι κωδικός τυχαίας επιλογής, δηλαδή διατίθεται σε παραπάνω από ένα σχέδια ή χρώματα. Δείτε τους όρους και προϋποθέσεις επιστροφών κωδικών τυχαίας επιλογής.\",\"DescriptionTabTitle\":\"Video\",\"VideoThumb\":\"VIDEO\",\"SafeClothingTitle\":\"ΠΙΣΤΟΠΟΙΗΣΗ\",\"SafeClothingTooltip\":\"Έχει ελεγχθεί για βλαβερές ουσίες\",\"SafeDishwasherTooltip\":\"Το προϊόν <span>δεν</span> είναι κατάλληλο για πλυντήριο πιάτων\",\"SafeMicrowaveTooltip\":\"Το προϊόν <span>δεν</span> είναι κατάλληλο για φούρνο μικροκυμάτων\",\"SuitableTitle\":\"ΚΑΤΑΛΛΗΛΟΤΗΤΑ\",\"WarehouseTooltip\":\"Επιλέξτε πρώτα μέγεθος για να δείτε την διαθεσιμότητα στα καταστήματα\",\"AvailabilityLabel\":\"Επιλέξτε μέγεθος για να δείτε την διαθεσιμότητα\",\"TabOneLabel\":\"Περιγραφή\",\"TabTwoLabel\":\"Μεταφορικά Έξοδα\",\"TabThreeLabel\":\"Επιστροφές\",\"TabFourLabel\":\"Χρόνοι Παράδοσης\",\"SizeGuideLabel\":\"Βρες το μέγεθος σου εδώ\",\"CanonicalLabel\":\"Δείτε τα όλα εδώ\",\"InstructionsPdfTitle\":\"Οδηγίες χρήσεως-συναρμολόγησης \",\"InstructionsPdfUrl\":\"Λήψη αρχείου PDF\"},\"SocialShares\":null,\"ProductReview\":null,\"WarehouseAvailabilities\":{\"__type\":\"Lighthouse.Lh3.FrontEnd.Root.Model.ProductWarehouseAvailabilityDataViewModel, Vendd.Website.Backend, Version=2020.4.0.72, Culture=neutral, PublicKeyToken=null\",\"GeoLocationWithZipCode\":true,\"IgnoreAvailability\":true,\"WarehouseResultsCount\":100000,\"PartnerGroupBy\":\"\",\"SelectStores\":false,\"EnableTrackingLocation\":false,\"WarehouseDistances\":[{\"__type\":\"Lighthouse.Lh3.FrontEnd.Root.Model.DistanceSetValueOptions, Vendd.Website.Backend, Version=2020.4.0.72, Culture=neutral, PublicKeyToken=null\",\"Value\":\"1000000\",\"Name\":\"χωρίς όριο\"}],\"CheckAvailabilityText\":\"ΑΝΑΖΗΤΗΣΗ ΚΑΤΑΣΤΗΜΑΤΩΝ\",\"CheckAvailabilityWithTrackingLocationText\":\"Search Warehouses Near My Location\",\"ShowWarehouseSearchText\":\"ΕΛΕΓΧΟΣ ΔΙΑΘΕΣΙΜΟΤΗΤΑΣ ΣΤΑ ΚΑΤΑΣΤΗΜΑΤΑ\",\"WarehouseHelpText\":\"Εισάγετε τον ΤΚ σας και δείτε τη διαθεσιμότητα στα καταστήματα Jumbo που βρίσκονται δίπλα σας.\",\"WarehouseHelpTextTwo\":\"Ελέγξτε την διαθεσιμότητα του προϊόντος στο κατάστημα της επιλογής σας:\",\"WarehouseTitle\":\"Διαθεσιμότητα σε κατάστημα\",\"WarehouseSearchText\":\"Εισάγετε ΤΚ\",\"WarehouseDistancesText\":\"Μέγιστη απόσταση\",\"WarehouseContinueShopping\":\"SELECT STORE & CONTINUE SHOPPING\",\"WarehouseProceedToCheckout\":\"SELECT STORE & PROCEED TO CHECKOUT\",\"LocationText\":\"Εισάγετε ΤΚ\",\"ErrorCalculatingDistancesText\":\"No calculation returned\",\"Configuration\":null,\"DebugInformation\":null}},\"LoaderAssemblyName\":\"Vendd.Website.Backend, Version=2020.4.0.72, Culture=neutral, PublicKeyToken=null\",\"LoaderClassName\":\"Lighthouse.Lh3.FrontEnd.Controls.Builtin.ProductDetailsLoader\",\"AdapterAssemblyName\":\"Vendd.Website.Backend, Version=2020.4.0.72, Culture=neutral, PublicKeyToken=null\",\"AdapterClassName\":\"Lighthouse.Lh3.FrontEnd.Root.Model.ProductDetailsAdapter\"}"
d_value = json.loads(d_value_data)
d_value["LoaderOptions"]["ParentProductId"] = 1409782
d_value["LoaderOptions"]["PathChecksum"] = 2027663942
d_value["LoaderOptions"]["SkuId"] = 1409921
d_value = json.dumps(d_value) 

data["serviceConfiguration"] = d_value
data = json.dumps(data)
response = requests.post('https://www.e-jumbo.gr/services/AdapterService.svc/GetData', headers=headers, params=params, data=data)

#NB. Original query string below. It seems impossible to parse and
#reproduce query strings 100% accurately so the one below is given
#in case the reproduced version is not "correct".
# response = requests.post('https://www.e-jumbo.gr/services/AdapterService.svc/GetData?lang=el&country=gr', headers=headers, cookies=cookies, data=data)
s_data_raw = response.json()
s_data = json.loads(s_data_raw["d"])
s_data["SkuCode"]

'1183758'

In [177]:
d_value_data = "{\"__type\":\"Lighthouse.Lh3.FrontEnd.Root.Model.AdapterParameters, Vendd.Website.Backend, Version=2020.4.0.72, Culture=neutral, PublicKeyToken=null\",\"LoaderOptions\":{\"__type\":\"Lighthouse.Lh3.FrontEnd.Controls.Builtin.ProductDetailsLoaderOptions, Vendd.Website.Backend, Version=2020.4.0.72, Culture=neutral, PublicKeyToken=null\",\"ParentProductId\":1445696,\"PathChecksum\":526038316,\"LanguageId\":1,\"SkuId\":1388210,\"SplitDimensions\":null,\"BundleItems\":[],\"ItemAttributes\":[],\"LanguageCode\":\"el\",\"CurrencyCode\":\"EUR\",\"SiteCode\":\"www-e-jumbo-gr\",\"Cacher\":null},\"AdapterOptions\":{\"__type\":\"Lighthouse.Lh3.FrontEnd.Root.Model.ProductDetailsBaseAdapter+ProductDetailsAdapterParams, Vendd.Website.Backend, Version=2020.4.0.72, Culture=neutral, PublicKeyToken=null\",\"IsDynamic\":false,\"EnableNoImage\":true,\"DimensionImageTypes\":[],\"SelectDefaultSku\":false,\"Actions\":2147483647,\"LazyLoadPrices\":false,\"AttributesStartIndex\":null,\"Articles\":{},\"Literals\":{\"QuantityText\":\"ΠΟΣΟΤΗΤΑ:\",\"VatText\":\"Στην τιμή συμπεριλαμβάνεται ο Φ.Π.A.\",\"ShopOnlyProductBtnText\":\"ΕΠΙΚΟΙΝΩΝΗΣΤΕ ΜΕ ΚΑΤΑΣΤΗΜΑ\",\"ShopOnlyProductBtnText2\":\"βρες το πλησιέστερο\",\"StoresLocatorUrl\":\"http://corporate.e-jumbo.gr/katastimata-jumbo/\",\"ShopOnlyProductBtnTextUrl\":\"εδώ\",\"IsBatteryRequiredText\":\"ΛΕΙΤΟΥΡΓΕΙ ΜΕ ΜΠΑΤΑΡΙΕΣ\",\"BatteryBuyText\":\"ΔΕΙΤΕ ΕΔΩ\",\"IsBatteryRequiredUrl\":\"/paichnidia/bataries-alkalikes/\",\"SafeProductTitle\":\"ΚΑΤΑΛΛΗΛΟΤΗΤΑ ΠΡΟΪΟΝΤΟΣ\",\"PriceText\":\"ΤΙΜΗ JUMBO:\",\"AvailabilityText\":\"Διαθεσιμότητα:\",\"OrderedWithIconClass\":\"rocket\",\"GalleryText\":\"Τοποθετήστε τον κέρσορα στη φωτογραφία για μεγέθυνση\",\"AssortmentCodesText\":\"Το προϊόν είναι κωδικός τυχαίας επιλογής, δηλαδή διατίθεται σε παραπάνω από ένα σχέδια ή χρώματα. Δείτε τους όρους και προϋποθέσεις επιστροφών κωδικών τυχαίας επιλογής.\",\"DescriptionTabTitle\":\"Video\",\"VideoThumb\":\"VIDEO\",\"SafeClothingTitle\":\"ΠΙΣΤΟΠΟΙΗΣΗ\",\"SafeClothingTooltip\":\"Έχει ελεγχθεί για βλαβερές ουσίες\",\"SafeDishwasherTooltip\":\"Το προϊόν <span>δεν</span> είναι κατάλληλο για πλυντήριο πιάτων\",\"SafeMicrowaveTooltip\":\"Το προϊόν <span>δεν</span> είναι κατάλληλο για φούρνο μικροκυμάτων\",\"SuitableTitle\":\"ΚΑΤΑΛΛΗΛΟΤΗΤΑ\",\"WarehouseTooltip\":\"Επιλέξτε πρώτα μέγεθος για να δείτε την διαθεσιμότητα στα καταστήματα\",\"AvailabilityLabel\":\"Επιλέξτε μέγεθος για να δείτε την διαθεσιμότητα\",\"TabOneLabel\":\"Περιγραφή\",\"TabTwoLabel\":\"Μεταφορικά Έξοδα\",\"TabThreeLabel\":\"Επιστροφές\",\"TabFourLabel\":\"Χρόνοι Παράδοσης\",\"SizeGuideLabel\":\"Βρες το μέγεθος σου εδώ\",\"CanonicalLabel\":\"Δείτε τα όλα εδώ\",\"InstructionsPdfTitle\":\"Οδηγίες χρήσεως-συναρμολόγησης \",\"InstructionsPdfUrl\":\"Λήψη αρχείου PDF\"},\"SocialShares\":null,\"ProductReview\":null,\"WarehouseAvailabilities\":{\"__type\":\"Lighthouse.Lh3.FrontEnd.Root.Model.ProductWarehouseAvailabilityDataViewModel, Vendd.Website.Backend, Version=2020.4.0.72, Culture=neutral, PublicKeyToken=null\",\"GeoLocationWithZipCode\":true,\"IgnoreAvailability\":true,\"WarehouseResultsCount\":100000,\"PartnerGroupBy\":\"\",\"SelectStores\":false,\"EnableTrackingLocation\":false,\"WarehouseDistances\":[{\"__type\":\"Lighthouse.Lh3.FrontEnd.Root.Model.DistanceSetValueOptions, Vendd.Website.Backend, Version=2020.4.0.72, Culture=neutral, PublicKeyToken=null\",\"Value\":\"1000000\",\"Name\":\"χωρίς όριο\"}],\"CheckAvailabilityText\":\"ΑΝΑΖΗΤΗΣΗ ΚΑΤΑΣΤΗΜΑΤΩΝ\",\"CheckAvailabilityWithTrackingLocationText\":\"Search Warehouses Near My Location\",\"ShowWarehouseSearchText\":\"ΕΛΕΓΧΟΣ ΔΙΑΘΕΣΙΜΟΤΗΤΑΣ ΣΤΑ ΚΑΤΑΣΤΗΜΑΤΑ\",\"WarehouseHelpText\":\"Εισάγετε τον ΤΚ σας και δείτε τη διαθεσιμότητα στα καταστήματα Jumbo που βρίσκονται δίπλα σας.\",\"WarehouseHelpTextTwo\":\"Ελέγξτε την διαθεσιμότητα του προϊόντος στο κατάστημα της επιλογής σας:\",\"WarehouseTitle\":\"Διαθεσιμότητα σε κατάστημα\",\"WarehouseSearchText\":\"Εισάγετε ΤΚ\",\"WarehouseDistancesText\":\"Μέγιστη απόσταση\",\"WarehouseContinueShopping\":\"SELECT STORE & CONTINUE SHOPPING\",\"WarehouseProceedToCheckout\":\"SELECT STORE & PROCEED TO CHECKOUT\",\"LocationText\":\"Εισάγετε ΤΚ\",\"ErrorCalculatingDistancesText\":\"No calculation returned\",\"Configuration\":null,\"DebugInformation\":null}},\"LoaderAssemblyName\":\"Vendd.Website.Backend, Version=2020.4.0.72, Culture=neutral, PublicKeyToken=null\",\"LoaderClassName\":\"Lighthouse.Lh3.FrontEnd.Controls.Builtin.ProductDetailsLoader\",\"AdapterAssemblyName\":\"Vendd.Website.Backend, Version=2020.4.0.72, Culture=neutral, PublicKeyToken=null\",\"AdapterClassName\":\"Lighthouse.Lh3.FrontEnd.Root.Model.ProductDetailsAdapter\"}"
d_value_data = d_value_data.replace(r'"ParentProductId":1445696',r'"ParentProductId":1409782')
print(d_value_data)

{"__type":"Lighthouse.Lh3.FrontEnd.Root.Model.AdapterParameters, Vendd.Website.Backend, Version=2020.4.0.72, Culture=neutral, PublicKeyToken=null","LoaderOptions":{"__type":"Lighthouse.Lh3.FrontEnd.Controls.Builtin.ProductDetailsLoaderOptions, Vendd.Website.Backend, Version=2020.4.0.72, Culture=neutral, PublicKeyToken=null","ParentProductId":1409782,"PathChecksum":526038316,"LanguageId":1,"SkuId":1388210,"SplitDimensions":null,"BundleItems":[],"ItemAttributes":[],"LanguageCode":"el","CurrencyCode":"EUR","SiteCode":"www-e-jumbo-gr","Cacher":null},"AdapterOptions":{"__type":"Lighthouse.Lh3.FrontEnd.Root.Model.ProductDetailsBaseAdapter+ProductDetailsAdapterParams, Vendd.Website.Backend, Version=2020.4.0.72, Culture=neutral, PublicKeyToken=null","IsDynamic":false,"EnableNoImage":true,"DimensionImageTypes":[],"SelectDefaultSku":false,"Actions":2147483647,"LazyLoadPrices":false,"AttributesStartIndex":null,"Articles":{},"Literals":{"QuantityText":"ΠΟΣΟΤΗΤΑ:","VatText":"Στην τιμή συμπεριλαμβάν

In [165]:
print(r'\"ParentProductId\":1445696')

\"ParentProductId\":1445696


In [176]:

d_value_data = "{\"__type\":\"Lighthouse.Lh3.FrontEnd.Root.Model.AdapterParameters, Vendd.Website.Backend, Version=2020.4.0.72, Culture=neutral, PublicKeyToken=null\",\"LoaderOptions\":{\"__type\":\"Lighthouse.Lh3.FrontEnd.Controls.Builtin.ProductDetailsLoaderOptions, Vendd.Website.Backend, Version=2020.4.0.72, Culture=neutral, PublicKeyToken=null\",\"ParentProductId\":1445696,\"PathChecksum\":526038316,\"LanguageId\":1,\"SkuId\":1388210,\"SplitDimensions\":null,\"BundleItems\":[],\"ItemAttributes\":[],\"LanguageCode\":\"el\",\"CurrencyCode\":\"EUR\",\"SiteCode\":\"www-e-jumbo-gr\",\"Cacher\":null},\"AdapterOptions\":{\"__type\":\"Lighthouse.Lh3.FrontEnd.Root.Model.ProductDetailsBaseAdapter+ProductDetailsAdapterParams, Vendd.Website.Backend, Version=2020.4.0.72, Culture=neutral, PublicKeyToken=null\",\"IsDynamic\":false,\"EnableNoImage\":true,\"DimensionImageTypes\":[],\"SelectDefaultSku\":false,\"Actions\":2147483647,\"LazyLoadPrices\":false,\"AttributesStartIndex\":null,\"Articles\":{},\"Literals\":{\"QuantityText\":\"ΠΟΣΟΤΗΤΑ:\",\"VatText\":\"Στην τιμή συμπεριλαμβάνεται ο Φ.Π.A.\",\"ShopOnlyProductBtnText\":\"ΕΠΙΚΟΙΝΩΝΗΣΤΕ ΜΕ ΚΑΤΑΣΤΗΜΑ\",\"ShopOnlyProductBtnText2\":\"βρες το πλησιέστερο\",\"StoresLocatorUrl\":\"http://corporate.e-jumbo.gr/katastimata-jumbo/\",\"ShopOnlyProductBtnTextUrl\":\"εδώ\",\"IsBatteryRequiredText\":\"ΛΕΙΤΟΥΡΓΕΙ ΜΕ ΜΠΑΤΑΡΙΕΣ\",\"BatteryBuyText\":\"ΔΕΙΤΕ ΕΔΩ\",\"IsBatteryRequiredUrl\":\"/paichnidia/bataries-alkalikes/\",\"SafeProductTitle\":\"ΚΑΤΑΛΛΗΛΟΤΗΤΑ ΠΡΟΪΟΝΤΟΣ\",\"PriceText\":\"ΤΙΜΗ JUMBO:\",\"AvailabilityText\":\"Διαθεσιμότητα:\",\"OrderedWithIconClass\":\"rocket\",\"GalleryText\":\"Τοποθετήστε τον κέρσορα στη φωτογραφία για μεγέθυνση\",\"AssortmentCodesText\":\"Το προϊόν είναι κωδικός τυχαίας επιλογής, δηλαδή διατίθεται σε παραπάνω από ένα σχέδια ή χρώματα. Δείτε τους όρους και προϋποθέσεις επιστροφών κωδικών τυχαίας επιλογής.\",\"DescriptionTabTitle\":\"Video\",\"VideoThumb\":\"VIDEO\",\"SafeClothingTitle\":\"ΠΙΣΤΟΠΟΙΗΣΗ\",\"SafeClothingTooltip\":\"Έχει ελεγχθεί για βλαβερές ουσίες\",\"SafeDishwasherTooltip\":\"Το προϊόν <span>δεν</span> είναι κατάλληλο για πλυντήριο πιάτων\",\"SafeMicrowaveTooltip\":\"Το προϊόν <span>δεν</span> είναι κατάλληλο για φούρνο μικροκυμάτων\",\"SuitableTitle\":\"ΚΑΤΑΛΛΗΛΟΤΗΤΑ\",\"WarehouseTooltip\":\"Επιλέξτε πρώτα μέγεθος για να δείτε την διαθεσιμότητα στα καταστήματα\",\"AvailabilityLabel\":\"Επιλέξτε μέγεθος για να δείτε την διαθεσιμότητα\",\"TabOneLabel\":\"Περιγραφή\",\"TabTwoLabel\":\"Μεταφορικά Έξοδα\",\"TabThreeLabel\":\"Επιστροφές\",\"TabFourLabel\":\"Χρόνοι Παράδοσης\",\"SizeGuideLabel\":\"Βρες το μέγεθος σου εδώ\",\"CanonicalLabel\":\"Δείτε τα όλα εδώ\",\"InstructionsPdfTitle\":\"Οδηγίες χρήσεως-συναρμολόγησης \",\"InstructionsPdfUrl\":\"Λήψη αρχείου PDF\"},\"SocialShares\":null,\"ProductReview\":null,\"WarehouseAvailabilities\":{\"__type\":\"Lighthouse.Lh3.FrontEnd.Root.Model.ProductWarehouseAvailabilityDataViewModel, Vendd.Website.Backend, Version=2020.4.0.72, Culture=neutral, PublicKeyToken=null\",\"GeoLocationWithZipCode\":true,\"IgnoreAvailability\":true,\"WarehouseResultsCount\":100000,\"PartnerGroupBy\":\"\",\"SelectStores\":false,\"EnableTrackingLocation\":false,\"WarehouseDistances\":[{\"__type\":\"Lighthouse.Lh3.FrontEnd.Root.Model.DistanceSetValueOptions, Vendd.Website.Backend, Version=2020.4.0.72, Culture=neutral, PublicKeyToken=null\",\"Value\":\"1000000\",\"Name\":\"χωρίς όριο\"}],\"CheckAvailabilityText\":\"ΑΝΑΖΗΤΗΣΗ ΚΑΤΑΣΤΗΜΑΤΩΝ\",\"CheckAvailabilityWithTrackingLocationText\":\"Search Warehouses Near My Location\",\"ShowWarehouseSearchText\":\"ΕΛΕΓΧΟΣ ΔΙΑΘΕΣΙΜΟΤΗΤΑΣ ΣΤΑ ΚΑΤΑΣΤΗΜΑΤΑ\",\"WarehouseHelpText\":\"Εισάγετε τον ΤΚ σας και δείτε τη διαθεσιμότητα στα καταστήματα Jumbo που βρίσκονται δίπλα σας.\",\"WarehouseHelpTextTwo\":\"Ελέγξτε την διαθεσιμότητα του προϊόντος στο κατάστημα της επιλογής σας:\",\"WarehouseTitle\":\"Διαθεσιμότητα σε κατάστημα\",\"WarehouseSearchText\":\"Εισάγετε ΤΚ\",\"WarehouseDistancesText\":\"Μέγιστη απόσταση\",\"WarehouseContinueShopping\":\"SELECT STORE & CONTINUE SHOPPING\",\"WarehouseProceedToCheckout\":\"SELECT STORE & PROCEED TO CHECKOUT\",\"LocationText\":\"Εισάγετε ΤΚ\",\"ErrorCalculatingDistancesText\":\"No calculation returned\",\"Configuration\":null,\"DebugInformation\":null}},\"LoaderAssemblyName\":\"Vendd.Website.Backend, Version=2020.4.0.72, Culture=neutral, PublicKeyToken=null\",\"LoaderClassName\":\"Lighthouse.Lh3.FrontEnd.Controls.Builtin.ProductDetailsLoader\",\"AdapterAssemblyName\":\"Vendd.Website.Backend, Version=2020.4.0.72, Culture=neutral, PublicKeyToken=null\",\"AdapterClassName\":\"Lighthouse.Lh3.FrontEnd.Root.Model.ProductDetailsAdapter\"}"
d_value = json.loads(d_value_data)
d_value["LoaderOptions"]["ParentProductId"] = 1409782
d_value["LoaderOptions"]["PathChecksum"] = 2027663942
d_value["LoaderOptions"]["SkuId"] = 1409921
d_value = str(d_value).replace("'",'\\"'))

{"__type": "Lighthouse.Lh3.FrontEnd.Root.Model.AdapterParameters, Vendd.Website.Backend, Version=2020.4.0.72, Culture=neutral, PublicKeyToken=null", "LoaderOptions": {"__type": "Lighthouse.Lh3.FrontEnd.Controls.Builtin.ProductDetailsLoaderOptions, Vendd.Website.Backend, Version=2020.4.0.72, Culture=neutral, PublicKeyToken=null", "ParentProductId": 1445696, "PathChecksum": 526038316, "LanguageId": 1, "SkuId": 1388210, "SplitDimensions": null, "BundleItems": [], "ItemAttributes": [], "LanguageCode": "el", "CurrencyCode": "EUR", "SiteCode": "www-e-jumbo-gr", "Cacher": null}, "AdapterOptions": {"__type": "Lighthouse.Lh3.FrontEnd.Root.Model.ProductDetailsBaseAdapter+ProductDetailsAdapterParams, Vendd.Website.Backend, Version=2020.4.0.72, Culture=neutral, PublicKeyToken=null", "IsDynamic": false, "EnableNoImage": true, "DimensionImageTypes": [], "SelectDefaultSku": false, "Actions": 2147483647, "LazyLoadPrices": false, "AttributesStartIndex": null, "Articles": {}, "Literals": {"QuantityText"

In [103]:
d_value

{'__type': 'Lighthouse.Lh3.FrontEnd.Root.Model.AdapterParameters, Vendd.Website.Backend, Version=2020.4.0.72, Culture=neutral, PublicKeyToken=null',
 'LoaderOptions': {'__type': 'Lighthouse.Lh3.FrontEnd.Controls.Builtin.ProductDetailsLoaderOptions, Vendd.Website.Backend, Version=2020.4.0.72, Culture=neutral, PublicKeyToken=null',
  'ParentProductId': 1409782,
  'PathChecksum': 2027663942,
  'LanguageId': 1,
  'SkuId': 1409921,
  'SplitDimensions': None,
  'BundleItems': [],
  'ItemAttributes': [],
  'LanguageCode': 'el',
  'CurrencyCode': 'EUR',
  'SiteCode': 'www-e-jumbo-gr',
  'Cacher': None},
 'AdapterOptions': {'__type': 'Lighthouse.Lh3.FrontEnd.Root.Model.ProductDetailsBaseAdapter+ProductDetailsAdapterParams, Vendd.Website.Backend, Version=2020.4.0.72, Culture=neutral, PublicKeyToken=null',
  'IsDynamic': False,
  'EnableNoImage': True,
  'DimensionImageTypes': [],
  'SelectDefaultSku': False,
  'Actions': 2147483647,
  'LazyLoadPrices': False,
  'AttributesStartIndex': None,
  'A

In [107]:

print(str(d_value).replace("'",'\\"'))

{\"__type\": \"Lighthouse.Lh3.FrontEnd.Root.Model.AdapterParameters, Vendd.Website.Backend, Version=2020.4.0.72, Culture=neutral, PublicKeyToken=null\", \"LoaderOptions\": {\"__type\": \"Lighthouse.Lh3.FrontEnd.Controls.Builtin.ProductDetailsLoaderOptions, Vendd.Website.Backend, Version=2020.4.0.72, Culture=neutral, PublicKeyToken=null\", \"ParentProductId\": 1409782, \"PathChecksum\": 2027663942, \"LanguageId\": 1, \"SkuId\": 1409921, \"SplitDimensions\": None, \"BundleItems\": [], \"ItemAttributes\": [], \"LanguageCode\": \"el\", \"CurrencyCode\": \"EUR\", \"SiteCode\": \"www-e-jumbo-gr\", \"Cacher\": None}, \"AdapterOptions\": {\"__type\": \"Lighthouse.Lh3.FrontEnd.Root.Model.ProductDetailsBaseAdapter+ProductDetailsAdapterParams, Vendd.Website.Backend, Version=2020.4.0.72, Culture=neutral, PublicKeyToken=null\", \"IsDynamic\": False, \"EnableNoImage\": True, \"DimensionImageTypes\": [], \"SelectDefaultSku\": False, \"Actions\": 2147483647, \"LazyLoadPrices\": False, \"AttributesStar

In [47]:
import requests

cookies = {
    'countrySelectorPopupOpened': 'true',
    '_gcl_au': '1.1.660232412.1623524496',
    '_fbp': 'fb.1.1623524496867.485701904',
    '_ga': 'GA1.2.2073830231.1623525700',
    'uid': '844ed38482c24916be65f6962a6f7c77',
    'newsletterpopup': 'true',
    'VisitorGUID': '4aac7d5a-4a57-4044-a795-887ab921ad6d',
    '_gid': 'GA1.2.972819527.1623839982',
    'sessionid': '2a1d90e7d6ae4d1f9edfc0138c2f316c',
    'ForceClearDeliveryPointLocalStorage': '1',
    '__atssc': 'skype%3B15',
    'ASP.NET_SessionId': 'ldkjeqpcw0potdgutsmis5b4',
    'ROUTEID.cd107d9706d71153bafd4ab15f1c6b5d': '.node3',
    'ConsentChecked': '{"userHasSetCookies":true,"functionalityCookies":true,"statisticCookies":true,"marketingCookies":true}',
    'SpPlList_1354735': 'LastProductId=1391620',
    '__atuvc': '81%7C24',
    '__atuvs': '60ca46e376b4f6d2007',
    'LastViewedProducts': '1445696,1409782,1394823,1374995',
    'CdnVisitorState': 'countryCode=gr&currencyCode=eur',
    'RT': 'sl=23&ss=1623864750276&tt=110148&obo=2&sh=1623869521803%3D23%3A2%3A110148%2C1623869496814%3D22%3A2%3A104500%2C1623869481773%3D21%3A2%3A99311%2C1623869470421%3D20%3A2%3A93665%2C1623869427124%3D19%3A2%3A87354&dm=www.e-jumbo.gr&si=7f1cf869-cefb-4479-b988-10df04ae4ba0&bcn=%2F%2F684fc539.akstat.io%2F&rl=1&ld=1623869521804',
    'exitIntentFlag': 'true',
    'PreviousCountryCode': 'GR',
    'VisitorState': 'siteId=1&countryCode=gr&languageCode=el&currencyCode=eur&partnerId=&futureDeliveryDateTime=&deliveryPointId=&userRoleIds=',
    'SpPlList_1354737': 'LastProductId=1394817',
    '_gat': '1',
}

headers = {
    'authority': 'www.google.co.in',
    'sec-ch-ua': '" Not;A Brand";v="99", "Google Chrome";v="91", "Chromium";v="91"',
    'accept': 'image/avif,image/webp,image/apng,image/svg+xml,image/*,*/*;q=0.8',
    'x-requested-with': 'XMLHttpRequest',
    'sec-ch-ua-mobile': '?0',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.106 Safari/537.36',
    'content-type': 'text/plain',
    'origin': 'https://www.e-jumbo.gr',
    'sec-fetch-site': 'cross-site',
    'sec-fetch-mode': 'no-cors',
    'sec-fetch-dest': 'image',
    'referer': 'https://www.e-jumbo.gr/',
    'accept-language': 'en-US,en;q=0.9,hi;q=0.8',
    'cookie': '__Secure-3PSID=-Qe_HfXmBTvemM-K7uXYlnd9d_6ylOUlJ6p6yyXGLbzRln9QWsZkVWoHQtYUk2l3vW6LAA.; __Secure-3PAPISID=rHLM80cHqTPjgriy/AWZCkAkgAKpQXZH3s; NID=216=PZsPtOmhlpvV0_7SFqjG1E8E9ctzsfPEvUUXgphqqLUU6Puyw02ult6S3gXuYjdYkcl4Trytaa2oHIc0uS4FZ74M4Na2Upoi00d26jkr9WFiFkYxXsDvqrYSmY8TbPg1pQTECSSBYoRz6HMdTSOYqmjCY4XBYNl9AUtu2wthEcInAZOcBlVJJI97T-9cnl4601zRyhhKpIAe28mVXY0wMXZzwcWmPJjS8so_63TLjZtJO22tZ9K6YvDxh6k9yc2XLijoKwmF-W0_NIgbaw0kL9xs1g',
    'content-length': '0',
    'Connection': 'keep-alive',
    'Accept': '*/*',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.106 Safari/537.36',
    'Content-Type': 'application/json',
    'Origin': 'https://www.e-jumbo.gr',
    'Sec-Fetch-Site': 'cross-site',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Dest': 'empty',
    'Referer': 'https://www.e-jumbo.gr/',
    'Accept-Language': 'en-US,en;q=0.9,hi;q=0.8',
    'Access-Control-Request-Method': 'POST',
    'Access-Control-Request-Headers': 'content-type',
    'x-client-data': 'CKO1yQEIkLbJAQiltskBCMS2yQEIqZ3KAQigoMsBCMCgywEI3PLLAQie+MsBCLP4ywE=',
}

params = (
    ('lang', 'el'),
    ('country', 'gr'),
)

data = {"serviceConfiguration":"{\"__type\":\"Lighthouse.Lh3.FrontEnd.Root.Model.AdapterParameters, Vendd.Website.Backend, Version=2020.4.0.72, Culture=neutral, PublicKeyToken=null\",\"LoaderOptions\":{\"__type\":\"Lighthouse.Lh3.FrontEnd.Controls.Builtin.ProductDetailsLoaderOptions, Vendd.Website.Backend, Version=2020.4.0.72, Culture=neutral, PublicKeyToken=null\",\"ParentProductId\":1445696,\"PathChecksum\":526038316,\"LanguageId\":1,\"SkuId\":1388211,\"SplitDimensions\":null,\"BundleItems\":[],\"ItemAttributes\":[],\"LanguageCode\":\"el\",\"CurrencyCode\":\"EUR\",\"SiteCode\":\"www-e-jumbo-gr\",\"Cacher\":null},\"AdapterOptions\":{\"__type\":\"Lighthouse.Lh3.FrontEnd.Root.Model.ProductDetailsBaseAdapter+ProductDetailsAdapterParams, Vendd.Website.Backend, Version=2020.4.0.72, Culture=neutral, PublicKeyToken=null\",\"IsDynamic\":false,\"EnableNoImage\":true,\"DimensionImageTypes\":[],\"SelectDefaultSku\":false,\"Actions\":2147483647,\"LazyLoadPrices\":false,\"AttributesStartIndex\":null,\"Articles\":{},\"Literals\":{\"QuantityText\":\"ΠΟΣΟΤΗΤΑ:\",\"VatText\":\"Στην τιμή συμπεριλαμβάνεται ο Φ.Π.A.\",\"ShopOnlyProductBtnText\":\"ΕΠΙΚΟΙΝΩΝΗΣΤΕ ΜΕ ΚΑΤΑΣΤΗΜΑ\",\"ShopOnlyProductBtnText2\":\"βρες το πλησιέστερο\",\"StoresLocatorUrl\":\"http://corporate.e-jumbo.gr/katastimata-jumbo/\",\"ShopOnlyProductBtnTextUrl\":\"εδώ\",\"IsBatteryRequiredText\":\"ΛΕΙΤΟΥΡΓΕΙ ΜΕ ΜΠΑΤΑΡΙΕΣ\",\"BatteryBuyText\":\"ΔΕΙΤΕ ΕΔΩ\",\"IsBatteryRequiredUrl\":\"/paichnidia/bataries-alkalikes/\",\"SafeProductTitle\":\"ΚΑΤΑΛΛΗΛΟΤΗΤΑ ΠΡΟΪΟΝΤΟΣ\",\"PriceText\":\"ΤΙΜΗ JUMBO:\",\"AvailabilityText\":\"Διαθεσιμότητα:\",\"OrderedWithIconClass\":\"rocket\",\"GalleryText\":\"Τοποθετήστε τον κέρσορα στη φωτογραφία για μεγέθυνση\",\"AssortmentCodesText\":\"Το προϊόν είναι κωδικός τυχαίας επιλογής, δηλαδή διατίθεται σε παραπάνω από ένα σχέδια ή χρώματα. Δείτε τους όρους και προϋποθέσεις επιστροφών κωδικών τυχαίας επιλογής.\",\"DescriptionTabTitle\":\"Video\",\"VideoThumb\":\"VIDEO\",\"SafeClothingTitle\":\"ΠΙΣΤΟΠΟΙΗΣΗ\",\"SafeClothingTooltip\":\"Έχει ελεγχθεί για βλαβερές ουσίες\",\"SafeDishwasherTooltip\":\"Το προϊόν <span>δεν</span> είναι κατάλληλο για πλυντήριο πιάτων\",\"SafeMicrowaveTooltip\":\"Το προϊόν <span>δεν</span> είναι κατάλληλο για φούρνο μικροκυμάτων\",\"SuitableTitle\":\"ΚΑΤΑΛΛΗΛΟΤΗΤΑ\",\"WarehouseTooltip\":\"Επιλέξτε πρώτα μέγεθος για να δείτε την διαθεσιμότητα στα καταστήματα\",\"AvailabilityLabel\":\"Επιλέξτε μέγεθος για να δείτε την διαθεσιμότητα\",\"TabOneLabel\":\"Περιγραφή\",\"TabTwoLabel\":\"Μεταφορικά Έξοδα\",\"TabThreeLabel\":\"Επιστροφές\",\"TabFourLabel\":\"Χρόνοι Παράδοσης\",\"SizeGuideLabel\":\"Βρες το μέγεθος σου εδώ\",\"CanonicalLabel\":\"Δείτε τα όλα εδώ\",\"InstructionsPdfTitle\":\"Οδηγίες χρήσεως-συναρμολόγησης \",\"InstructionsPdfUrl\":\"Λήψη αρχείου PDF\"},\"SocialShares\":null,\"ProductReview\":null,\"WarehouseAvailabilities\":{\"__type\":\"Lighthouse.Lh3.FrontEnd.Root.Model.ProductWarehouseAvailabilityDataViewModel, Vendd.Website.Backend, Version=2020.4.0.72, Culture=neutral, PublicKeyToken=null\",\"GeoLocationWithZipCode\":true,\"IgnoreAvailability\":true,\"WarehouseResultsCount\":100000,\"PartnerGroupBy\":\"\",\"SelectStores\":false,\"EnableTrackingLocation\":false,\"WarehouseDistances\":[{\"__type\":\"Lighthouse.Lh3.FrontEnd.Root.Model.DistanceSetValueOptions, Vendd.Website.Backend, Version=2020.4.0.72, Culture=neutral, PublicKeyToken=null\",\"Value\":\"1000000\",\"Name\":\"χωρίς όριο\"}],\"CheckAvailabilityText\":\"ΑΝΑΖΗΤΗΣΗ ΚΑΤΑΣΤΗΜΑΤΩΝ\",\"CheckAvailabilityWithTrackingLocationText\":\"Search Warehouses Near My Location\",\"ShowWarehouseSearchText\":\"ΕΛΕΓΧΟΣ ΔΙΑΘΕΣΙΜΟΤΗΤΑΣ ΣΤΑ ΚΑΤΑΣΤΗΜΑΤΑ\",\"WarehouseHelpText\":\"Εισάγετε τον ΤΚ σας και δείτε τη διαθεσιμότητα στα καταστήματα Jumbo που βρίσκονται δίπλα σας.\",\"WarehouseHelpTextTwo\":\"Ελέγξτε την διαθεσιμότητα του προϊόντος στο κατάστημα της επιλογής σας:\",\"WarehouseTitle\":\"Διαθεσιμότητα σε κατάστημα\",\"WarehouseSearchText\":\"Εισάγετε ΤΚ\",\"WarehouseDistancesText\":\"Μέγιστη απόσταση\",\"WarehouseContinueShopping\":\"SELECT STORE & CONTINUE SHOPPING\",\"WarehouseProceedToCheckout\":\"SELECT STORE & PROCEED TO CHECKOUT\",\"LocationText\":\"Εισάγετε ΤΚ\",\"ErrorCalculatingDistancesText\":\"No calculation returned\",\"Configuration\":null,\"DebugInformation\":null}},\"LoaderAssemblyName\":\"Vendd.Website.Backend, Version=2020.4.0.72, Culture=neutral, PublicKeyToken=null\",\"LoaderClassName\":\"Lighthouse.Lh3.FrontEnd.Controls.Builtin.ProductDetailsLoader\",\"AdapterAssemblyName\":\"Vendd.Website.Backend, Version=2020.4.0.72, Culture=neutral, PublicKeyToken=null\",\"AdapterClassName\":\"Lighthouse.Lh3.FrontEnd.Root.Model.ProductDetailsAdapter\"}"}

data = json.dumps(data)

response = requests.post('https://www.e-jumbo.gr/services/AdapterService.svc/GetData', headers=headers, params=params, cookies=cookies, data=data)

#NB. Original query string below. It seems impossible to parse and
#reproduce query strings 100% accurately so the one below is given
#in case the reproduced version is not "correct".
response = requests.post('https://www.e-jumbo.gr/services/AdapterService.svc/GetData?lang=el&country=gr', headers=headers, cookies=cookies, data=data)


In [48]:
response.text

'{"d":"{\\"ParentProductId\\":1445696,\\"SelectedProductId\\":1388211,\\"PathChecksum\\":526038316,\\"LanguageId\\":1,\\"Code\\":\\"S1004507\\",\\"Name\\":\\"Σουτιέν Εγκυμοσύνης Nude Λευκή Δαντέλα\\",\\"SkuCode\\":\\"1178258\\",\\"SelectedSkuContentProviderCode\\":\\"1178258\\",\\"CheckAvailabilityText\\":null,\\"ShowWarehouseSearchText\\":null,\\"WarehouseHelpText\\":null,\\"WarehouseHelpTextTwo\\":null,\\"WarehouseTitle\\":null,\\"WarehouseSearchText\\":null,\\"Category\\":\\"Εσώρουχα Εγκυμοσύνης\\",\\"SmallDescription\\":\\"<p>Σουτιέν εγκυμοσύνης και θηλασμού nude με λευκή δαντέλα, ρυθμιζόμενα ραντάκια, χωρίς μπανέλες με πλαστικά κλιπ που διευκολύνουν τον θηλασμό.<\\/p>\\\\r\\\\n<p>Σύνθεση: 95% Βαμβάκι - 5% Ελαστάνη<\\/p>\\",\\"ExtendedDescription\\":\\"\\",\\"ManufacturerUrl\\":\\"\\",\\"ManufacturerName\\":\\"\\",\\"ManufacturerImage\\":null,\\"AvailabilityActionId\\":3,\\"MinQuantity\\":1.0,\\"MaxQuantity\\":2269.00000,\\"Stock\\":2269.00000,\\"Quanta\\":1.0,\\"MeasurementUnit\\"

In [181]:
statesAndCapitals = {
                     'Gujarat' : 'Gandhinagar',
                     'Maharashtra' : 'Mumbai',
                     'Rajasthan' : 'Jaipur',
                     'Bihar' : 'Patna'
                    }
                      
print('List Of given states:\n')
  
# Iterating over keys
for state in statesAndCapitals.keys():
    print(state)

List Of given states:

Gujarat
Maharashtra
Rajasthan
Bihar
